In [15]:
import pickle
import pandas as pd
import sklearn

In [16]:
sklearn.__version__

'1.6.1'

In [17]:
year = 2023
month = 4

input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/yellow_tripdata_{year:04d}-{month:02d}.parquet'

In [18]:
!mkdir output

mkdir: cannot create directory ‘output’: File exists


In [19]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

/home/manasa/Manasa-mlops/mlzoomcamp/lib/python3.10/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/manasa/Manasa-mlops/mlzoomcamp/lib/python3.10/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [20]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [21]:
df = read_data(input_file)
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [22]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration,ride_id
0,1,2023-04-01 00:14:49,2023-04-01 00:45:01,2.0,4.90,1.0,N,48,223,1,...,3.5,0.5,6.0,0.00,1.0,39.9,2.5,0.00,30.200000,2023/04_0
1,2,2023-04-01 00:00:24,2023-04-01 00:56:19,1.0,21.89,2.0,N,132,43,2,...,0.0,0.5,0.0,6.55,1.0,81.8,2.5,1.25,55.916667,2023/04_1
2,1,2023-04-01 00:03:50,2023-04-01 00:14:42,2.0,1.30,1.0,N,148,113,1,...,3.5,0.5,2.0,0.00,1.0,18.4,2.5,0.00,10.866667,2023/04_2
3,1,2023-04-01 00:53:18,2023-04-01 01:01:28,1.0,1.50,1.0,N,249,79,1,...,3.5,0.5,1.0,0.00,1.0,16.0,2.5,0.00,8.166667,2023/04_3
4,2,2023-04-01 00:07:00,2023-04-01 00:17:16,2.0,1.49,1.0,N,158,246,1,...,1.0,0.5,1.0,0.00,1.0,17.4,2.5,0.00,10.266667,2023/04_4


In [23]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [24]:
y_pred

array([16.13130203, 32.26481598, 12.25557264, ..., 12.30118266,
       12.5560282 , 11.39399629], shape=(3199715,))

### Q1. Standard deviation

In [25]:
y_pred.std()

np.float64(6.353996941249663)

### Q2. Preparing the output

In [26]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

In [27]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [28]:
!ls -lh output

total 129M
-rw-r--r-- 1 manasa manasa 66M Apr 10 17:11 yellow_tripdata_2023-03.parquet
-rw-r--r-- 1 manasa manasa 64M Apr 10 17:52 yellow_tripdata_2023-04.parquet
